In [ ]:
请你写出一个sql语句查询每个用户最近一天登录的日子，用户的名字，以及用户用的设备的名字，并且查询结果按照user的name升序排序，
"""Find date, user name, and name of client of most recent login for each user, order by user name ascending"""

In [ ]:
"""SQL table creation and data insertion"""
drop table if exists login;
drop table if exists user;
drop table if exists client;

CREATE TABLE `login` (
`id` int(4) NOT NULL,
`user_id` int(4) NOT NULL,
`client_id` int(4) NOT NULL,
`date` date NOT NULL,
PRIMARY KEY (`id`));

CREATE TABLE `user` (
`id` int(4) NOT NULL,
`name` varchar(32) NOT NULL,
PRIMARY KEY (`id`));

CREATE TABLE `client` (
`id` int(4) NOT NULL,
`name` varchar(32) NOT NULL,
PRIMARY KEY (`id`));

INSERT INTO login VALUES
(1,2,1,'2020-10-12'),
(2,3,2,'2020-10-12'),
(3,1,3,'2020-10-12'),
(4,2,2,'2020-10-13'),
(5,3,2,'2020-10-13');

INSERT INTO user VALUES
(1,'tm'),
(2,'fh'),
(3,'wangchao');

INSERT INTO client VALUES
(1,'pc'),
(2,'ios'),
(3,'anroid'),
(4,'h5');

In [ ]:
"""SQL solution"""
SELECT u.name, c.name, l.date
FROM login AS l, user AS u, client AS c,
(SELECT user_id, MAX(date) AS ld
FROM login
GROUP BY user_id) AS d
WHERE l.user_id = u.id
AND l.client_id = c.id
AND l.user_id = d.user_id
AND l.date = d.ld
ORDER BY u.name

In [1]:
"""pandas dataframe creation"""
import pandas as pd

login = pd.DataFrame([[1,2,1,'2020-10-12'],
                      [2,3,2,'2020-10-12'],
                      [3,1,3,'2020-10-12'],
                      [4,2,2,'2020-10-13'],
                      [5,3,2,'2020-10-13']],
                     columns=['id', 'user_id', 'client_id', 'date'])

user = pd.DataFrame([[1,'tm'],
                     [2,'fh'],
                     [3,'wangchao']],
                    columns=['id', 'name'])

client = pd.DataFrame([[1,'pc'],
                       [2,'ios'],
                       [3,'android'],
                       [4,'h5']],
                      columns=['id','name'])
login.head()

,id,user_id,client_id,date
0,1,2,1,2020-10-12
1,2,3,2,2020-10-12
2,3,1,3,2020-10-12
3,4,2,2,2020-10-13
4,5,3,2,2020-10-13


In [2]:
user.head()

,id,name
0,1,tm
1,2,fh
2,3,wangchao


In [3]:
client.head()

,id,name
0,1,pc
1,2,ios
2,3,android
3,4,h5


In [11]:
# find the most recent login for each user
islast = login.groupby('user_id')['date'].transform(max) == login['date']
last_login = login[islast]
last_login

,id,user_id,client_id,date
2,3,1,3,2020-10-12
3,4,2,2,2020-10-13
4,5,3,2,2020-10-13


In [12]:
last_login = pd.merge(last_login, user, left_on='user_id', right_on='id', how='inner', suffixes=['','_user'])
last_login = last_login.drop(columns=['id_user'])
last_login

,id,user_id,client_id,date,name
0,3,1,3,2020-10-12,tm
1,4,2,2,2020-10-13,fh
2,5,3,2,2020-10-13,wangchao


In [13]:
last_login = pd.merge(last_login, client, left_on='client_id', right_on='id', how='inner', suffixes=['','_client'])
last_login = last_login.drop(columns=['id_client'])
last_login

,id,user_id,client_id,date,name,name_client
0,3,1,3,2020-10-12,tm,android
1,4,2,2,2020-10-13,fh,ios
2,5,3,2,2020-10-13,wangchao,ios


In [14]:
last_login[['name','name_client','date']]

,name,name_client,date
0,tm,android,2020-10-12
1,fh,ios,2020-10-13
2,wangchao,ios,2020-10-13
